In [3]:
import json
from IPython.display import display, Image
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import Image as WidgetImage

In [69]:
def load_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)
class label_tool():
    def __init__(self, json_name, start, end):
        self.current_index = start
        self.data = load_data(json_name) 
        self.updated_data = []
        self.end = end
        self.start = start
    def display_content(self):
        clear_output(wait=True)
        if self.current_index < self.end:
            entry = self.data[self.current_index]

            # Load images from files and display them side by side
            with open(entry['image_1'], "rb") as file:
                image_input = file.read()
            with open(entry['image_2'], "rb") as file:
                image_output = file.read()
            with open(entry['image_diff'], "rb") as file:
                image_diff = file.read()
            
            

            print(f' Image ID {self.current_index}')
            
            box_1 = widgets.HBox([
                WidgetImage(value=image_input, format='png', width=500, height=500),
                WidgetImage(value=image_output, format='png', width=500, height=500)
            ])
            display(box_1)
            box_2 = widgets.HBox([
                WidgetImage(value=image_diff, format='png', width=500, height=500),
            ])
            display(box_2)
    
            # Display instruction
            print("Differences:", entry['difference'])
            
            image_2_object_list = set(entry['image_2_object_list'].split(','))
            image_1_object_list = set(entry['image_1_object_list'].split(','))
            image_2_object_list.update(image_1_object_list)
            #print(image_2_object_list, image_1_object_list)
            # Display editable text area for GPT_option
            text_area = widgets.Textarea(
                value=','.join(list(image_2_object_list)),
                placeholder='Type something',
                description='GPT Option:',
                disabled=False,
                layout={'width': '100%'}
            )
            
            # Display editable text area for GPT_option
            correct_area = widgets.Textarea(
                value='',
                placeholder='Type something',
                description='Answer:',
                disabled=False,
                layout={'width': '100%'}
            )
            
            display(text_area)
            display(correct_area)

            # Button to submit changes
            submit_button = widgets.Button(description="Save Changes")
            
#             # Button to double check changes
#             check_button = widgets.Button(description="Checked & Save Changes")
            
            def on_submit_clicked(b):
                # Update the entry with the new GPT_option and save it to the updated_data list
                
                entry['Modified_option'] = text_area.value
                entry['Answer'] = correct_area.value

                    
                self.updated_data.append(entry)
                self.current_index += 1
                clear_output(wait=True)
                self.display_next_button()  # Display the "Show Next" button again
                

            submit_button.on_click(on_submit_clicked)
#             def back_function(button):
#                 self.updated_data.pop()
#                 print('remove last pair')
#             back_buttons = widgets.Button(description="Back")
#             back_buttons.on_click(back_function)
#             display(back_buttons)
            display(submit_button)
        else:
            print("No more entries to display. Please call save_updated_data() to save results.")
    
    def save_updated_data(self):
        with open(f'surveillance_{self.start}_{self.end}.json', 'w') as file:
            json.dump(self.updated_data, file)
            
    def display_next_button(self):
        next_button = widgets.Button(description="Show Next")
        next_button.on_click(lambda b: self.display_content())
        display(next_button)

In [72]:
labeler = label_tool('surveillance_source.json', 20,25)

In [73]:
labeler.display_next_button()

 Image ID 24


Differences: ['the people where taken out of the photo']


Textarea(value=' brick building, bare trees, red bicycles,black car, yellow poles, walking people.,Yellow pole…

Textarea(value='', description='Answer:', layout=Layout(width='100%'), placeholder='Type something')

Button(description='Save Changes', style=ButtonStyle())

In [74]:
labeler.updated_data

[{'image_1': 'spot-the-diff/pair_images/test/12868/12868.png',
  'image_2': 'spot-the-diff/pair_images/test/12868/12868_2.png',
  'image_diff': 'spot-the-diff/pair_images/test/12868/12868_diff.jpg',
  'difference': ['four additional people are in the parking lot compared to the left image'],
  'image_1_object_list': 'yellow poles, leafless trees, walking people',
  'image_2_object_list': 'Walking people, red bricks, leafless trees, gray sidewalk, yellow lines.',
  'gpt_extract_objects_gt': 'people',
  'Modified_option': ' leafless trees, walking people, gray sidewalk, yellow lines.,Walking people, red bricks,yellow poles',
  'Answer': ''},
 {'image_1': 'spot-the-diff/pair_images/test/2698/2698.png',
  'image_2': 'spot-the-diff/pair_images/test/2698/2698_2.png',
  'image_diff': 'spot-the-diff/pair_images/test/2698/2698_diff.jpg',
  'difference': ['white suv 5th from the left is missing in parking lot'],
  'image_1_object_list': 'white SUV, black cars, grey sedan, white van, wet pavement

In [24]:
labeler.save_updated_data()